## Khởi tạo Spark

In [1]:
import findspark
findspark.init()

import pyspark
findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import count

spark = (SparkSession
         .builder
         .appName("Heart Disease")
         .getOrCreate())

## Đọc và load tập dữ liệu Boston housing

In [72]:
heartDF = (spark.read
            .option("HEADER", True)
            .option("inferSchema", True)
            .csv("C:/Users/Phung Huy Hoang/Downloads/framingham.csv")
           )

heartDF.show()

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|  BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|   1| 39|        4|            0|         0|     0|              0|           0|       0|    195|106.0| 70.0|26.97|       80|     77|         0|
|   0| 46|        2|            0|         0|     0|              0|           0|       0|    250|121.0| 81.0|28.73|       95|     76|         0|
|   1| 48|        1|            1|        20|     0|              0|           0|       0|    245|127.5| 80.0|25.34|       75|     70|         0|
|   0| 61|        3|            1|        30|     0|              0|           1|       0|    225|150.0| 95.0|28.58|       6

## Tập dữ liệu Framingham Heart Disease
`male`: 1 là `nam`, 0 là `nữ`

`age`: Độ tuổi

`education`: Trình độ giáo dục

`currentSmoker`: Có đang hút thuốc hay không

`cigsPerDay`: Số điếu thuốc mỗi ngày

`BPMeds`: Có đang dùng thuốc huyết áp hay không

`prevalentStroke`: Trước đó có bị đột quỵ hay không

`prevalentHyp`: Có bị tăng huyết áp hay không

`diabetes`: Có bị tiểu đường hay không

`totChol`: Tổng mức cholesterol

`sysBP`: Huyết áp tâm thu

`diaBP`: Huyết áp tâm trương

`BMI`: Chỉ số khối cơ thể

`heartRate`: Nhịp tim

`glucose`: Lượng đường trong máu

Biến dự báo:

`TenYearCHD`: Nguy cơ mắc bệnh tim mạch vành trong 10 năm

## Xem các loại biến trong tập dữ liệu

In [49]:
heartDF.dtypes

[('male', 'int'),
 ('age', 'int'),
 ('education', 'string'),
 ('currentSmoker', 'int'),
 ('cigsPerDay', 'string'),
 ('BPMeds', 'string'),
 ('prevalentStroke', 'int'),
 ('prevalentHyp', 'int'),
 ('diabetes', 'int'),
 ('totChol', 'string'),
 ('sysBP', 'double'),
 ('diaBP', 'double'),
 ('BMI', 'string'),
 ('heartRate', 'string'),
 ('glucose', 'string'),
 ('TenYearCHD', 'int')]

Các biến được lưu trong file CSV dưới dạng số. Tuy nhiên khi đọc vào Spark, một số biến lại trở thành dạng `string` bởi vì trong biến có chứa giá trị `'NA'`.

Những biến nào chứa giá trị `'NA'` sẽ thay bằng giá trị ... (chưa nghĩ ra)

In [73]:
from pyspark.sql.functions import *
heartDF = heartDF.withColumn('education', regexp_replace('education', 'NA', '-9999')).\
    withColumn('education', col("education").cast('int')).\
    withColumn('cigsPerDay', regexp_replace('cigsPerDay', 'NA', '-9999')).\
    withColumn('cigsPerDay', col("cigsPerDay").cast('int')).\
    withColumn('BPMeds', regexp_replace('BPMeds', 'NA', '-9999')).\
    withColumn('BPMeds', col("BPMeds").cast('int')).\
    withColumn('totChol', regexp_replace('totChol', 'NA', '-9999')).\
    withColumn('totChol', col("totChol").cast('int')).\
    withColumn('BMI', regexp_replace('BMI', 'NA', '-9999')).\
    withColumn('BMI', col("BMI").cast('double')).\
    withColumn('heartRate', regexp_replace('heartRate', 'NA', '-9999')).\
    withColumn('heartRate', col("heartRate").cast('int')).\
    withColumn('glucose', regexp_replace('glucose', 'NA', '-9999')).\
    withColumn('glucose', col("glucose").cast('int'))

heartDF.dtypes

[('male', 'int'),
 ('age', 'int'),
 ('education', 'int'),
 ('currentSmoker', 'int'),
 ('cigsPerDay', 'int'),
 ('BPMeds', 'int'),
 ('prevalentStroke', 'int'),
 ('prevalentHyp', 'int'),
 ('diabetes', 'int'),
 ('totChol', 'int'),
 ('sysBP', 'double'),
 ('diaBP', 'double'),
 ('BMI', 'double'),
 ('heartRate', 'int'),
 ('glucose', 'int'),
 ('TenYearCHD', 'int')]

## Chia dữ liệu thành train/test set

In [74]:
(trainDF, testDF) = heartDF.randomSplit([.8, .2], seed=1)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|  BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|   1| 39|        4|            0|         0|     0|              0|           0|       0|    195|106.0| 70.0|26.97|       80|     77|         0|
|   0| 46|        2|            0|         0|     0|              0|           0|       0|    250|121.0| 81.0|28.73|       95|     76|         0|
|   1| 48|        1|            1|        20|     0|              0|           0|       0|    245|127.5| 80.0|25.34|       75|     70|         0|
|   0| 61|        3|            1|        30|     0|              0|           1|       0|    225|150.0| 95.0|28.58|       6

## Biến đổi train data theo định dạng của Spark

In [75]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['male','age','education','currentSmoker',\
                                       'cigsPerDay','BPMeds','prevalentStroke',\
                                       'prevalentHyp','diabetes','totChol','sysBP',\
                                       'diaBP','BMI','heartRate', 'glucose'],
                            outputCol='features') 
assembler_train = assembler.transform(trainDF)
final_train = assembler_train.select('features','TenYearCHD')
final_train.show(3)

+--------------------+----------+
|            features|TenYearCHD|
+--------------------+----------+
|[0.0,32.0,2.0,1.0...|         0|
|(15,[1,2,9,10,11,...|         0|
|[0.0,33.0,2.0,1.0...|         0|
+--------------------+----------+
only showing top 3 rows



# 1. Decision Tree

## Huấn luyện mô hình Decision Tree Regressor trên train data

**1.1** Huấn luyện mô hình trên `final_train` với `DecisionTreeRegressor` với `labelCol` là `'TenYearCHD'` và `featuresCol` là `'features'`

In [70]:
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(labelCol = 'TenYearCHD', featuresCol = 'features')
dtModel = dt.fit(final_train)

Py4JJavaError: An error occurred while calling o738.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 27, 192.168.70.1, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(VectorAssembler$$Lambda$2458/1145457780: (struct<male_double_VectorAssembler_a0a845897161:double,age_double_VectorAssembler_a0a845897161:double,education_double_VectorAssembler_a0a845897161:double,currentSmoker_double_VectorAssembler_a0a845897161:double,cigsPerDay_double_VectorAssembler_a0a845897161:double,BPMeds_double_VectorAssembler_a0a845897161:double,prevalentStroke_double_VectorAssembler_a0a845897161:double,prevalentHyp_double_VectorAssembler_a0a845897161:double,diabetes_double_VectorAssembler_a0a845897161:double,totChol_double_VectorAssembler_a0a845897161:double,sysBP:double,diaBP:double,BMI:double,heartRate_double_VectorAssembler_a0a845897161:double,glucose_